# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json

irp_client = IRPClient()

### Configurations

In [ ]:
server_name = "databridge-1"
edm_name = f"CBHU_Automated" # ensure unique within IRP
treaty_name = "CBHU_Treaty"
edm_data_version = "22"
portfolio_name = "CBHU_Automated" # ensure unique within the edm
geohaz_version = "22.0"
accounts_file_name = "accountsfile.csv"
locations_file_name = "locationsfile.csv"
mapping_file_name = "mapping.json"
analysis_job_name = "CBHU Analysis"
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"
analysis_tag_name = "CBHU_TAG"
rdm_name = "CBHU_Automated_RDM"

### Create EDM(s)

In [ ]:
job_ids = irp_client.edm.submit_create_edm_jobs(
    [
        {
            "edm_name": edm_name,
            "server_name": server_name
        }
    ]
)
print(json.dumps(irp_client.job.poll_risk_data_job_batch_to_completion(job_ids), indent=2))

### Create Portfolio(s)

In [ ]:
portfolio_ids = irp_client.portfolio.create_portfolios(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "portfolio_number": portfolio_name,
            "description": f"{portfolio_name} created via IRP Notebook Framework"
        }
    ]
)
print(f"Created {len(portfolio_ids)} portfolios with IDs: {portfolio_ids}")

### MRI Import

In [ ]:
# edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
# if (len(edms) != 1):
#     raise Exception(f"Expected 1 EDM with name {edm_name}, found {len(edms)}")
# exposure_id = edms[0]['exposureId']

# portfolios = irp_client.portfolio.search_portfolios(exposure_id=exposure_id, filter=f"portfolioName=\"{portfolio_name}\"")
# if (len(portfolios) != 1):
#     raise Exception(f"Expected 1 portfolio with name {portfolio_name}, found {len(portfolios)}")
# portfolio_uri = portfolios[0]['uri']

# # create_folder_response = irp_client.mri_import.create_import_folder(
# #     'MRI', 
# #     'csv', 
# #     ['accountsFile', 'locationsFile', 'mappingFile'])
# create_folder_response = irp_client.mri_import.create_import_folder(
#     'MRI', 
#     'csv', 
#     ['accountsFile', 'locationsFile'])

# irp_client.mri_import.upload_file_to_s3(
#     create_folder_response['uploadDetails']['accountsFile'],
#     accounts_file_name,
#     create_folder_response['uploadDetails']['accountsFile']['fileUri'].split('/')[-1])
# irp_client.mri_import.upload_file_to_s3(
#     create_folder_response['uploadDetails']['locationsFile'],
#     locations_file_name,
#     create_folder_response['uploadDetails']['locationsFile']['fileUri'].split('/')[-1])
# # irp_client.mri_import.upload_file_to_s3(
# #     create_folder_response['uploadDetails']['mappingFile'],
# #     mapping_file_name)

# job_id = irp_client.mri_import.create_import_job(create_folder_response, portfolio_uri)
# job_response = irp_client.client.poll_risk_data_job_to_completion(job_id)
# print(json.dumps(job_response, indent=2))


In [ ]:
import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_name=portfolio_name,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json"
)

print(json.dumps(import_response, indent=2))

### Create Treaty

In [ ]:
treaty_ids = irp_client.treaty.create_treaties(
    [
        {
            "edm_name": edm_name,
            "treaty_name": treaty_name,
            "treaty_number": treaty_name,
            "treaty_type": "Working Excess",
            "per_risk_limit": 30000000.00,
            "occurrence_limit": 90000000.00,
            "attachment_point": 2000000.00,
            "inception_date": "2025-01-01T00:00:00.000Z",
            "expiration_date": "2030-01-01T00:00:00.000Z",
            "currency_name": "US Dollar",
            "attachment_basis": "Losses Occurring",
            "attachment_level": "Location",
            "pct_covered": 100.0,
            "pct_placed": 95.00,
            "pct_share": 100.00,
            "pct_retention": 100.00,
            "premium": 0.00,
            "num_reinstatements": 99,
            "pct_reinstatement_charge": 0.00,
            "aggregate_limit": 0.00,
            "aggregate_deductible": 0.00,
            "priority": 1
        }
    ]
)
print(f"Created {len(treaty_ids)} with IDs: {treaty_ids}")

### Upgrade EDM Version

In [ ]:
upgrade_job_ids = irp_client.edm.submit_upgrade_edm_data_version_jobs(
    [
        {
            "edm_name": edm_name,
            "edm_version": edm_data_version
        }
    ]
)
print(json.dumps(irp_client.edm.poll_data_version_upgrade_job_batch_to_completion(upgrade_job_ids), indent=2))

### GeoHaz

In [ ]:
geohaz_job_ids = irp_client.portfolio.submit_geohaz_jobs(
    [
        {
            "edm_name": edm_name,
            "portfolio_name": portfolio_name,
            "version": geohaz_version,
            "hazard_eq": True,
            "hazard_ws": True
        }
    ]
)
print(json.dumps(irp_client.portfolio.poll_geohaz_job_batch_to_completion(geohaz_job_ids), indent=2))

### Execute Multiple Analyses

In [ ]:
analysis_job_ids = irp_client.analysis.submit_portfolio_analysis_jobs(
    [
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2023 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2023 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        },
        {
            "edm_name": edm_name,
            "job_name": "CBHU 2025 Rates",
            "portfolio_name": portfolio_name,
            "analysis_profile_name": analysis_profile_name,
            "output_profile_name": output_profile_name,
            "event_rate_scheme_name": "RMS 2025 Stochastic Event Rates",
            "treaty_names": [treaty_name],
            "tag_names": [analysis_tag_name]
        }
    ]
)
analysis_batch_response = irp_client.analysis.poll_analysis_job_batch_to_completion(analysis_job_ids)
print(json.dumps(analysis_batch_response, indent=2))

### Group Analyses

In [ ]:
analysis_names = ["CBHU 2023 Rates", "CBHU 2025 Rates"]
grouping_job_ids = irp_client.analysis.submit_analysis_grouping_jobs(
    [
        {
            "group_name": "CBHU_Group",
            "analysis_names": analysis_names
        }
    ]
)
grouping_response = irp_client.analysis.poll_analysis_grouping_job_batch_to_completion(grouping_job_ids)
print(json.dumps(grouping_response, indent=2))

### Export Analyses to RDM

In [ ]:
analysis_names = ["CBHU Group", "CBHU 2023 Rates", "CBHU 2025 Rates"]
rdm_export_response = irp_client.rdm.export_analyses_to_rdm(
    server_name=server_name,
    rdm_name=rdm_name,
    analysis_names=analysis_names
)
print(json.dumps(rdm_export_response, indent=2))

### Delete EDM

In [ ]:
print(json.dumps(irp_client.edm.delete_edm(edm_name), indent=2))